# 1-Librairies

In [ ]:
# Dataset
import kagglehub

# Standard libraries
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import numpy as np
import pandas as pd
import tqdm as notebook_tqdm

# tensorflow libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import callbacks

# keras libraries
from keras.optimizers import Adam

# scikit-learn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Suppression des warnings
import warnings
warnings.filterwarnings("ignore")

# 2-Dataset

In [ ]:
# Téléchargement du dataset depuis Kaggle
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Chemin d'accés du dossier du Dataset:", path)

In [ ]:
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

## 2.1-Dataset Entrainement

In [ ]:
# Création de train_df à partir du dossier Training
train_images = []
train_labels = []

# Utiliser explicitement la liste des noms de classes pour éviter le conflit avec la variable labels (one-hot)
for category in class_names:
    category_path = os.path.join(path, "Training", category)
    for img_name in os.listdir(category_path):
        train_images.append(os.path.join(category_path, img_name))
        train_labels.append(category)

# Création du DataFrame train_df
train_df = pd.DataFrame({"image_path": train_images, "label": train_labels})

X_train = train_df["image_path"].values
y_train = train_df["label"].values

## 2.2-Dataset Test

In [ ]:
# Création de test_df à partir du dossier Testing
test_images = []
test_labels = []

for category in class_names:
    category_path = os.path.join(path, "Testing", category)
    for img_name in os.listdir(category_path):
        test_images.append(os.path.join(category_path, img_name))
        test_labels.append(category)

test_df = pd.DataFrame({"image_path": test_images, "label": test_labels})

X_test = test_df["image_path"].values
y_test = test_df["label"].values

## 2.3-Dataset de validation

In [ ]:
# Creation d'un dataset de validation
X_train, X_val, y_train, y_val = train_test_split(
    train_df["image_path"].to_numpy(),
    train_df["label"].to_numpy(),
    train_size=0.7,
    random_state=42,
    stratify=train_df["label"],
)
val_df = pd.DataFrame({'image_path': X_val, 'label': y_val})

# 3-Preprocessing

## 3.1-Normaliser le dataset

In [ ]:
# Taille du batch
batch_size = 32

# Taille des images
# Note: Les images sont redimensionnées à 224x224 pixels pour correspondre à l'entrée du modèle
img_size = (224,224)  # Taille des images (hauteur, largeur)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

# Mode de couleur des images
color_mode = 'rgb'  # Couleur des images (RGB)

## 3.2-Création des générateurs d'images

In [ ]:
_gen = ImageDataGenerator(
    horizontal_flip=True,  # Retourner les images horizontalement
    rescale=1./255,
)
# genérateur pour les données d'entraînement
train_gen = _gen.flow_from_dataframe(
    train_df, x_col='image_path', y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode=color_mode,
    batch_size=batch_size
)
# Générateur pour les données de validation
valid_gen = _gen.flow_from_dataframe(
    val_df, x_col='image_path', y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode=color_mode,
    shuffle=True,
    batch_size=batch_size
)
# Générateur pour les données de test
test_gen = _gen.flow_from_dataframe(
    test_df, x_col='image_path', y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode=color_mode,
    shuffle=False,
    batch_size=batch_size
)


## 3.3-Visualisation du preprocessing

In [ ]:
def show_batch(dataset, title="Batch d'images", num_images=9, figsize=(12, 12)):

    plt.figure(figsize=figsize)
    plt.suptitle(title, fontsize=16, fontweight='bold')
    
    for images, labels in dataset.take(1):
        for i in range(min(num_images, len(images))):
            ax = plt.subplot(3, 3, i + 1)
            
            # Récupérer et préparer l'image pour l'affichage
            image = images[i].numpy()
            
            # Si l'image est normalisée [-1, 1], la ramener à [0, 1]
            if image.min() < 0:
                image = (image + 1) / 2
            
            # Si les valeurs sont > 1, normaliser
            if image.max() > 1:
                image = image / 255.0
            
            plt.imshow(image)
            plt.title(f"{class_names[labels[i]]}", fontsize=12, pad=10)
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

print("Fonction de visualisation créée")

# Affichage d'un batch d'images originales du générateur d'entraînement
print("Aperçu des images originales du dataset:")
print("Note: Images redimensionnées à 224x224 pour permettre l'affichage en batch")
batch_images, batch_labels = next(train_gen)
plt.figure(figsize=(12, 6))
plt.suptitle("Images Originales (redimensionnées pour affichage)", fontsize=16, fontweight='bold')
for i in range(10):
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(batch_images[i])
    plt.title(f"{class_names[np.argmax(batch_labels[i])]}", fontsize=12, pad=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

# Affichage après préprocessing (sans augmentation)
# La fonction preprocess_image n'est pas définie car ImageDataGenerator gère déjà le préprocessing.
# Si vous souhaitez visualiser les images prétraitées, utilisez simplement le générateur train_gen.
# Exemple : afficher un batch du générateur d'entraînement (déjà prétraité)
batch_images, batch_labels = next(train_gen)
plt.figure(figsize=(12, 6))
plt.suptitle("Images Après Préprocessing (224x224, normalisées)", fontsize=16, fontweight='bold')
for i in range(10):
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(batch_images[i])
    plt.title(f"{class_names[np.argmax(batch_labels[i])]}", fontsize=12, pad=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

# 4-CNN (Réseau Convolutif)

## 4.1-Création du modèle

In [ ]:
# Création du modèle CNN
def create_cnn_model(img_shape, num_classes=4):

    model=Sequential([

        # Premier bloc convolutionnel
        Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Deuxième bloc convolutionnel
        Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Troisième bloc convolutionnel
        Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.5),

        # Classification
        Flatten(),

        Dense(512,activation = "relu"),
        Dense(num_classes, activation = "sigmoid")
    ])

    return model

# Compilation CNN
def compile_models(cnn_model):
    compile_config= {
        'optimizer': Adam(learning_rate=0.001, decay=1e-6),
        'loss': 'categorical_crossentropy',
        'metrics': ['accuracy', 'Precision', 'Recall']
    }

    # Compilation du modèle
    cnn_model.compile(**compile_config)

    return cnn_model

# Création du modèle CNN avec les bonnes dimensions et nombre de classes
cnn_model = create_cnn_model(img_shape, num_classes=4)

# Compiler le modele CNN
cnn_model = compile_models(cnn_model)

# Affichage des architectures
print("\n=== ARCHITECTURE CNN ===")
cnn_model.summary()

## 4.2-Entrainement du modele

In [ ]:
# Création des callbacks
def create_callbacks():
    callbacks_list = [

        # Sauvegarde du meilleur modèle
        callbacks.ModelCheckpoint(
            './models/model_cnn_US2_v1.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),

        # Early stopping pour éviter l'overfitting
        callbacks.EarlyStopping(
            monitor='loss',
            patience=1,
            restore_best_weights=True,
            verbose=1,
        ),

        # TensorBoard pour le monitoring
        callbacks.TensorBoard(
            './logs',
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    return callbacks_list

# Création des callbacks
callbacks_list = create_callbacks()

# Entraînement du modèle
print("=== Début de l'entraînement ===")

history = cnn_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=20,
    callbacks=callbacks_list,
    verbose=1
)

print("=== Entraînement terminé ===")

In [ ]:
history.history.keys()

## 4.3-Evaluation du modèle

In [ ]:
# Evaluation du modèle
loss, accuracy, precision, recall= cnn_model.evaluate(test_gen)
print(f"Test Loss: {loss:0.4f}")
print(f"Test Accuracy: {accuracy:0.4f}")
print(f"Test Precision: {precision:0.4f}")
print(f"Test Recall: {recall:0.4f}")

## 4.4-Visualisation du modèle

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
tr_per = history.history['precision']
tr_recall = history.history['recall']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
val_per = history.history['val_precision']
val_recall = history.history['val_recall']

index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
index_precision = np.argmax(val_per)
per_highest = val_per[index_precision]
index_recall = np.argmax(val_recall)
recall_highest = val_recall[index_recall]

Epochs = [i + 1 for i in range(len(tr_acc))]
loss_label = f'Best epoch = {str(index_loss + 1)}'
acc_label = f'Best epoch = {str(index_acc + 1)}'
per_label = f'Best epoch = {str(index_precision + 1)}'
recall_label = f'Best epoch = {str(index_recall + 1)}'


plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')

# affichage des métriques Loss d'entraînement et de validation
plt.subplot(2, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label='Training loss')
plt.plot(Epochs, val_loss, 'g', label='Validation loss')
plt.scatter(index_loss + 1, val_lowest, s=150, c='blue', label=loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Affichage des métriques Accuracy d'entraînement et de validation
plt.subplot(2, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label='Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label='Validation Accuracy')
plt.scatter(index_acc + 1, acc_highest, s=150, c='blue', label=acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Affichage des métriques de précision
plt.subplot(2, 2, 3)
plt.plot(Epochs, tr_per, 'r', label='Precision')
plt.plot(Epochs, val_per, 'g', label='Validation Precision')
plt.scatter(index_precision + 1, per_highest, s=150, c='blue', label=per_label)
plt.title('Precision and Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)

# Affichage des métriques de rappel
plt.subplot(2, 2, 4)
plt.plot(Epochs, tr_recall, 'r', label='Recall')
plt.plot(Epochs, val_recall, 'g', label='Validation Recall')
plt.scatter(index_recall + 1, recall_highest, s=150, c='blue', label=recall_label)
plt.title('Recall and Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)

plt.suptitle('Métriques d_entrainement sur le modèle', fontsize=16)
plt.show()

In [ ]:
# Matrice de Confusion et rapport de classification

def test_model(model, test_gen):
    # Charger les poids du meilleur modèle sauvegardé
    model.load_weights('./models/model_cnn_US2_v1.keras')
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_gen.classes
    test_acc = 100 * np.sum(predicted_classes == true_classes) / len(true_classes)
    
    print(f"Test Accuracy: {test_acc:.2f}%\n")
    
    print("Classification Report:\n")
    print(classification_report(true_classes, predicted_classes, target_names=class_names))

    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

print("\nLoading best model for testing...")
try:
    cnn_model
except NameError:
    print("Erreur : 'cnn_model' n'est pas défini. Veuillez vérifier l'entraînement du modèle.")
else:
    test_model(cnn_model, test_gen)

# 5-Prédiction

In [ ]:
# Visualisation des prédictions
def visualize_predictions(model, generator, num_samples=8):
    
    # Charger les poids du modèle
    model.load_weights('model_cnn_US2.v1.keras')
    
    # Obtenir un batch d'images et leurs étiquettes
    x, y_true = next(generator)
    y_pred = model.predict(x)
    
    # Convertir les étiquettes vraies et prédites en classes 
    true_classes = np.argmax(y_true, axis=1)
    pred_classes = np.argmax(y_pred, axis=1)
    
    # Trouver les indices des classes correctes et incorrectes
    correct_indices = np.where(true_classes == pred_classes)[0]
    incorrect_indices = np.where(true_classes != pred_classes)[0]
    
    # Selectioner un nombre équilibré d'images correctes et incorrectes
    num_correct = min(num_samples//2, len(correct_indices))
    num_incorrect = min(num_samples//2, len(incorrect_indices))
    
    selected_correct = np.random.choice(correct_indices, num_correct, replace=False)
    selected_incorrect = np.random.choice(incorrect_indices, num_incorrect, replace=False)
    selected_indices = np.concatenate([selected_correct, selected_incorrect])
    
    # Visualisation
    plt.figure(figsize=(16, 10))
    gs = gridspec.GridSpec(2, num_samples//2, height_ratios=[4, 4])
    
    for i, idx in enumerate(selected_indices):
        
        plt.subplot(gs[i])
        img = x[idx]
        if img.shape[-1] == 1:  # si l'image est en niveaux de gris
            img = np.concatenate([img]*3, axis=-1)
        plt.imshow(img)
        plt.axis('off')
        
        # Determine si la prédiction est correcte
        is_correct = true_classes[idx] == pred_classes[idx]
        
        # Ajouter les étiquettes de classe
        true_label = class_names[int(true_classes[idx])]
        pred_label = class_names[int(pred_classes[idx])]
        
        title_color = 'green' if is_correct else 'red'
        plt.title(f"True: {true_label}\nPred: {pred_label}", 
                color=title_color, fontsize=10)
        
        # Ajouter la confiance de la prédiction
        confidence = np.max(y_pred[idx]) * 100
        plt.text(5, 15, f"{confidence:.1f}%", 
                bbox=dict(facecolor='white', alpha=0.8), 
                fontsize=8)
    
    plt.tight_layout()
    plt.show()

# Appeler la fonction pour visualiser les prédictions
print("Visualizing predictions...")
visualize_predictions(cnn_model, test_gen, num_samples=8)